# Training for All Regions

In [1]:
intialregions=['5j','INC_ge6j3b','INC_ge6jge4b']
regions=['INC_5j3b','INC_5jge4b','INC_ge6j3b','INC_ge6jge4b']

invertTestTrain=False
hpmasses=[200,225,250,275,300,350,400,500,600,700,800,900,1000,1200,1400,1600,1800,2000]

In [2]:
#! rm -rf HplusML pandas_INC_*.h5
#! wget https://jglatzer.web.cern.ch/jglatzer/hpml/pandas_INC_ge6j3b.h5
#! wget https://jglatzer.web.cern.ch/jglatzer/hpml/pandas_INC_ge6jge4b.h5
#! wget https://jglatzer.web.cern.ch/jglatzer/hpml/pandas_INC_5j3b.h5
#! wget https://jglatzer.web.cern.ch/jglatzer/hpml/pandas_INC_5jge4b.h5

## Plot BDTS

In [5]:
import pandas as pd
import HpTrainingFrame
from sklearn.metrics import roc_auc_score
from keras.models import model_from_json
from joblib import load

roc_aucs_bdt={}
roc_aucs_mpbdt={}
roc_aucs_nn={}
roc_aucs_mpnn={}


for region in regions[:1]:
    roc_aucs_bdt[region]=[]
    roc_aucs_mpbdt[region]=[]
    roc_aucs_nn[region]=[]
    roc_aucs_mpnn[region]=[]

    subregion=region
    if "5j" in region:
        subregion="5j"
    print(region)
    df_mc=pd.read_hdf('pandas_'+region+'.h5', region)
    
    for invertTestTrain in [False,True][:1]:
        roc_aucs_bdt_vec=[]
        roc_aucs_mpbdt_vec=[]
        roc_aucs_nn_vec=[]
        roc_aucs_mpnn_vec=[]

        datalabel="train2mod0"
        if invertTestTrain:
            datalabel="train2mod1"
            X_tmp=X_train
            y_tmp=y_train
            w_tmp=w_train
            X_train=X_test
            y_train=y_test
            w_train=w_test
            X_test=X_tmp
            y_test=y_tmp
            w_test=w_tmp
        
        # mass-parameterised BDT
        mpclfwss=load('models/massparameterisedBDT_'+subregion+'_'+datalabel+'_pipe.joblib')
        mpclf=load('models/massparameterisedBDT_'+subregion+'_'+datalabel+'_bdt.joblib')
        
        # mass-parameterised NN
        mpwss=load('models/massparameterisedNN3_'+subregion+'_'+datalabel+'_pipe.joblib') 
        json_file = open('models/massparameterisedNN3_'+subregion+'_'+datalabel+'_architecture.h5')
        loaded_model_json = json_file.read()
        json_file.close()
        nnmpmodel = model_from_json(loaded_model_json)
        nnmpmodel.load_weights('models/massparameterisedNN3_'+subregion+'_'+datalabel+'_weights.h5')
            
        #calculate values
        for hpmass in hpmasses[:1]:
            print("Region=",region,"invertTestTrain=", invertTestTrain,"H+ mass=",hpmass)
            
            htf=HpTrainingFrame.HpTrainingFrame(df_mc)
            X_train, X_test, X_eval, y_train, y_test,y_eval, w_train, w_test, w_eval=htf.prepare(hpmass=hpmass, region=region)

            X_testmass=X_test.copy()
            X_testmass["hpmass"]=hpmass


            from sklearn.metrics import roc_auc_score            
            
            #evaluate BDT
            clf=load('models/standardBDT_'+str(hpmass)+'_'+subregion+'_'+datalabel+'.joblib')
            roc_auc_bdt=1.-roc_auc_score(y_test, clf.predict_proba(X_test)[:,1], sample_weight=w_test)
            roc_aucs_bdt_vec.append(roc_auc_bdt)
            
            #evaluate NN       
            wsstrain=load('models/standardNN_'+str(hpmass)+'_'+subregion+'_'+datalabel+'_pipe.joblib').steps[1][1]
            print("pipe:",wsstrain)
            json_file = open('models/standardNN_'+str(hpmass)+'_'+subregion+'_'+datalabel+'_architecture.h5')
            loaded_model_json = json_file.read()
            json_file.close()
            nnmodel = model_from_json(loaded_model_json)
            nnmodel.load_weights('models/standardNN_'+str(hpmass)+'_'+subregion+'_'+datalabel+'_weights.h5')
            print(X_test.shape,X_test.mean(), X_test.var())
            X_testnn,y_testnn,w_testnn =wsstrain.transform(X_test.values,y_test.values,w_test.values)
            print(X_testnn.shape,X_testnn.mean(axis=0),X_testnn.var(axis=0))
            roc_auc_nn=1.-roc_auc_score(y_test, nnmodel.predict(X_testnn), sample_weight=w_test)
            roc_aucs_nn_vec.append(roc_auc_nn)
            
            #evaluate mass-parameterised BDT
            #X_testmpbdt,y_testmpbdt,w_testmpbdt=mpclfwss.transform(X_testmass,y_test,w_test)
            #roc_auc_mpbdt=1.-roc_auc_score(y_test, mpclf.predict_proba(X_testmpbdt)[:,1], sample_weight=w_test)
            #roc_aucs_mpbdt_vec.append(roc_auc_mpbdt)
            
            #evaluate mass-parameterised NN
            #X_testmpnn,y_testmpnn,w_testmpnn=mpclfwss.transform(X_testmass,y_test,w_test)
            #roc_auc_mpnn=1.-roc_auc_score(y_test, nnmpmodel.predict(X_testmpnn.values), sample_weight=w_test)
            #roc_aucs_mpnn_vec.append(roc_auc_mpnn)            
            
        roc_aucs_bdt[region].append(roc_aucs_bdt_vec)
        roc_aucs_mpbdt[region].append(roc_aucs_bdt_vec)
        roc_aucs_nn[region].append(roc_aucs_nn_vec)
        roc_aucs_mpnn[region].append(roc_aucs_mpnn_vec)
                
        #plot now
print("BDT:",roc_aucs_bdt)
print("MPBDT:",roc_aucs_mpbdt)
print("NN:",roc_aucs_nn)
print("MPNN:",roc_aucs_mpnn)

INC_5j3b
Region= INC_5j3b invertTestTrain= False H+ mass= 200
pipe: WeightedStandardScaler(copy=True, with_mean=True, with_std=True)
(184525, 14) nJets                    5.000000
nBTags_70                3.000000
pT_jet1             147817.814401
Mbb_MindR_70        102102.289728
pT_jet5              35079.023672
H1_all                   0.303689
dRbb_avg_70              2.240910
dRlepbb_MindR_70         2.246280
Muu_MindR_70        138714.673146
HT_jets             399387.466121
Mbb_MaxPt_70        136098.366801
Mbb_MaxM_70         231951.846600
Mjjj_MaxPt          238125.603451
Centrality_all           0.599355
dtype: float64 nJets               0.000000e+00
nBTags_70           0.000000e+00
pT_jet1             7.690846e+09
Mbb_MindR_70        3.876098e+09
pT_jet5             1.315241e+08
H1_all              6.210794e-02
dRbb_avg_70         2.312447e-01
dRlepbb_MindR_70    7.579067e-01
Muu_MindR_70        1.665158e+10
HT_jets             2.973113e+10
Mbb_MaxPt_70        9.877167e+09


/workspace/HplusML/trainall/HpMLFeatureNormalisation.py:79: RuntimeWarning: invalid value encountered in true_divide
  X /= self.scale_


ValueError: Error when checking input: expected dense_1_input to have 2 dimensions, but got array with shape (184525, 1, 13)

In [ ]:
print("BDT:",roc_aucs_bdt)
print("MPBDT:",roc_aucs_mpbdt)
print("NN:",roc_aucs_nn)
print("MPNN:",roc_aucs_mpnn)
import matplotlib.pyplot as plt
region="INC_5j3b"
plt.plot(hpmasses,roc_aucs_bdt[region][0],label="BDT")
plt.plot(hpmasses,roc_aucs_mpbdt[region][0],label="MP BDT")
plt.plot(hpmasses,roc_aucs_nn[region][0],label="NN")
plt.plot(hpmasses,roc_aucs_mpnn[region][0],label="MP NN")
plt.legend()
plt.show()